# Chirp Sequence Isolation and Chirp Estimation

## Step 0: Prerequisites

### 0.1: Brief Jupyter Lab Tutorial

#### Notebook Kernel
The **kernel** of a Jupyter notebook specifies the language in which the notebook is run.

You need to use a Julia kernel for this notebook. If you haven't installed `IJulia`, please do so ([installation instructions](https://julialang.github.io/IJulia.jl/stable/manual/installation/)).
If `IJulia` is installed properly, you should have the option to choose a Julia kernel (e.g. "Julia 1.9.3"), or Jupyter may select it automatically. You can see which kernel is selected by looking at the upper-right corner of this page. You can change the kernel by clicking on the kernel name.

#### "Help" Tab
The "Help" tab on the top bar has a lot of good resources. Here are some key ones for JupyterLab:

- _"Show Keyboard Shortcuts"_: a list of all keyboard shortcuts
- _"Show Contextual Help"_: opens a panel to the side that will show documentation for functions when you click on the function name in a code cell.
- _"Jupyter Reference"_ and _"JupyterLab Reference"_: detailed documentation for using Jupyter notebooks. You probably don't need to look at this, but it might be a good read.

#### Keyboard Shortcuts
You can go to "Help" > "Show Keyboard Shortcuts" to see a list of keyboard shortcuts. Here's an abbreviated list, as well as some instructions for mouse-based notebook navigation:

- Click on a code cell and press `SHIFT+ENTER` to run the cell. For the most part, the cells should be run in sequential order.
  
- Use `CTRL+ENTER` instead of `SHIFT+ENTER` if you don't want to move to the next cell after running the current one.
  
- When a code cell is being run, it will have `[*]` to the left of it. When execution is finished, the asterisk is replaced by a number, e.g. `[1]`. The number counts up for each cell you run (the first cell run will have `[1]`, the second cell run will have `[2]`, etc. If you run a cell multiple times, its number will keep on incrementing).
  
- Left click the white space to the left of a cell to select the cell without editing its code.
  
    - Doing this will reveal a blue vertical bar to the left of the cell. Clicking the bar will collapse the contents of the cell, and you can click the bar again to un-collapse the cell contents.
      
    - Clicking directly to the left of a cell output will put it into scroll mode. You can click the space to the left of the output again to make the output not scrollable.
      
- When you have a cell selected (but you're not editing it), you can press `a` to make a new cell above it and `b` to make a new cell below it.
  
- Press `SHIFT+L` (while not editing a cell) to show line numbers in the code cells.
  
- If the last line of a cell doesn't have a semicolon at the end, its value will be displayed below the cell. For instance, run the below cell

In [ ]:
A = π

How do you get that pi symbol? Type `\pi` in a code cell and press `TAB`. Then, wait for a little popup that says `π irrational {:π}` and either press `TAB` or `ENTER`.

### 0.2: Julia Resources

If you want Julia documentation or a tutorial, look at [this page](https://julialang.org/learning/) of the official Julia website.

**For documentation on the Batlab-specific code**, look at [**this page**](https://nsagan271.github.io/BatlabJuliaUtils/build/index.html).
This also includes tips on common issues you may encounter.

### 0.3: Have you set up the data?
If not, here are some instructions:
1. Go to the `matlab_utils` subdirectory of `batlab-chirp-processing` and use `tdms_to_mat.m` to convert some bat audio TDMS files to MAT files.
   - It is recommended you start off with the `Pu166_01` and `Pu166_02` data from 10/05/2021, as this is what was used to test the notebook. Other experiments, like `Gr116`, will require you to try out different values of the parameters in the cells marked with `### CONFIGURABLE PARAMETERS`.
   - To start off, you can save audio files to the `batlab-chirp-processing/data` directory. You can also use any other directory of your choosing.
2. Copy all mic position files to `data`.
4. Make a subdirectory of `data` called `centroid` and copy all relevant centroid data files to that directory.

### 0.4: Load the packages

To install Julia packages:
1. Go into the Julia command line. You can do this by running the `julia` command from any terminal, or by running the `julia` application.
2. Hit the `]` key.
3. Type `add Plots`, and then press Enter.
4. Do the same for `Printf`, `MAT`, `Statistics`, `Roots`, `DataInterpolations`, `DSP`, and `WAV`.
5. Press `Backspace` to exit package mode, and `CTRL+d` to exit the Julia command line.

You will only have to install these packages once. Also, if you get an error like
```
ArgumentError: Package HelloWorld not found in current path,
```
you will have to repeat the above process, but for whichever package is missing.

In [ ]:
import Pkg;
Pkg.develop(path="../BatlabJuliaUtils")

In [ ]:
using BatlabJuliaUtils

In [ ]:
## Some other packages
## If this cell errors, please do the following:
### 1. Go into the Julia command line.
### 2. Press the "]" key.
### 3. Run the command `add <Package>`, for all packages listed here. For
###    example, `add Plots`, then `add Printf`, etc.
using Plots;
using Printf;
using MAT;
using Statistics;
using Roots;
using DataInterpolations;
using DSP;
using WAV;

### 0.5: Define where the audio and centroid data is

The following cells assume that you have added `Pu166_02` data to the `data` directory, as described in step **0.3**. To use different data, replace the filenames below.

In [ ]:
AUDIO_FILENAME = "../data/Pu166_02.mat";
CENTROID_FILENAME = "../data/centroid/Pu166_002_centroidxyz.mat";
MIC_POSITION_FILENAME = "../data/mic_positions_fall2021.mat";

CENTROID_VARIABLE_NAME = collect(keys(matread(CENTROID_FILENAME)))[1];
MIC_POSITION_VARIABLE_NAME = collect(keys(matread(MIC_POSITION_FILENAME)))[1];

In [ ]:
(@printf "Do the variable names for these MAT files look right?\n\tFor the centroid file: \"%s\",\n\tand for the mic position file: \"%s\"" CENTROID_VARIABLE_NAME MIC_POSITION_VARIABLE_NAME)

If the variable names for the MAT files don't look right, then uncomment and run the following two cells:

In [ ]:
# println("The variable names of the centroid file are:\n", keys(matread(CENTROID_FILENAME)), "\nand the MAT file looks like")
# centroids = matread(CENTROID_FILENAME)

In [ ]:
# println("The variable names of the mic position file are:\n", keys(matread(MIC_POSITION_FILENAME)), "\nand the MAT file looks like")
# mic_locations = matread(MIC_POSITION_FILENAME)

## Step 1: Read in the audio data

The following cell reads the microphone data and creates a matrix `y`, where each column corresponds to a different microphone.

In [ ]:
y = readmicdata(AUDIO_FILENAME);

**Plot the data**. The first argument is the range of indices (of the microphone data) to plot. For instance, `plotmicdata(1:100_000, y)` plots the first 100,000 samples of microphone data, for each microphone.

In [ ]:
plotmicdata(1:100_000, y)

### Get a sample of noise for estimating the SNR

In [ ]:
noise_sample_idxs = getnoisesampleidxs(y);
noise_sample = y[noise_sample_idxs, :];
plotmicdata(noise_sample_idxs, y, title="Noise Sample")

You can see that the noise is not zero-mean, so let's subtract out the mean of the noise.

In [ ]:
y = y .- mean(noise_sample; dims=1);
noise_sample = noise_sample .- mean(noise_sample; dims=1);

## Step 2: Determine which time segments have signal (for each mic)

### First, estimate the signal-to-noise ratio (SNR)

The next cell estimates the signal-to-noise ratio for every timepoint of the microphone data and saves it into a matrix `snr`, where `snr` has the same dimensions as `y`.
The SNR is measured in decibels. In general, an SNR of 30 dB is considered decent signal strength, and higher SNR means that the signal is stronger.

_The next cell might take a minute or two._

In [ ]:
snr = estimatesnr(y, noise_sample, window_size=128);

In [ ]:
plotmicdata(1:100_000, snr, title="Signal-to-Noise Ratio", ylabel="Decibels")

You can see that the peaks of the SNR correspond to the vocalizations in the microphone data we plotted earlier.

### What does the SNR look like during Buzz phase?

When the bat is not in buzz phase, it is fairly easy to separate out chirps from segments that are just noise. However, the data is the messiest during buzz phase, so let's use the buzz phase to validate that chirp detection algorithms from future sections of this notebook work.

**When do the buzz phase chirps happen?**

You can either find buzz phase indices by playing around with the first argument  `plotmicdata`.
Also, the following cell can give some good starting points (it makes crude estimates of chirp onsets, as heard by the microphones. Then it finds the times where chirp onsets are very close in time):

In [ ]:
estimatebuzzphase(snr, buzz_phase_chirp_separation_ms=20);

Set the value of `INTERESTING_IDXS` below to the audio indices for the buzz phase, or any other indices of the microphone data you want to examine.

Note that if the interval is too long, plotting will be slow and it will be hard to see what's going on.

In [ ]:
# INTERESTING_IDXS = 1440297:1511542; # buzz phase for PU166_01
INTERESTING_IDXS = 1313803:1405018; # buzz phase for PU166_02

In [ ]:
plotmicdata(INTERESTING_IDXS, snr, title="Signal-to-Noise Ratio", ylabel="Decibels")

### Separate Chirps from Noise
The goal of this section is to find, approximately, where the vocalizations appear in each microphone's signal.
We first find "high-SNR regions", which are regions where the SNR is above a certain threshold. We want each high-SNR region to contain _exactly one vocalization_ (and maybe some echos after the vocalization),

This is done by setting thresholds on the SNR:
- The SNR estimate is quite noisy, and we want to find contiguous regions that have signal. So, we apply what is known as a "maximum filter" to the SNR. For every timepoint, this filter returns the highest SNR in a `MAXFILTER_LENGTH` radius around that timepoint.
- 30 db is considered "good SNR" for speech, so it makes sense to set the cutoff for what regions contain signal vs. noise around there.
- In order to make sure that the high-SNR regions have good enough signal strength, we impose another constraint: at some point within the "signal region", the SNR must pass another, higher threshold. We describe how this threshold is determined in the comment at the end of the below cell.

In [ ]:
### CONFIGURABLE PARAMETERS ###
SIGNAL_THRESH = 30
MAXFILTER_LENGTH_MS = 0.1 # If a point is at most MAXFILTER_LENGTH_MS milliseconds
                          # from a point where the SNR is over SIGNAL_THRESH, then
                          # that point is considered part of a signal region.
MAXFILTER_LENGTH = Int64(round(MAXFILTER_LENGTH_MS / 1000 * FS));

## The threshold imposed on the peak of a signal region has several components.
## First, it must not be below MIN_PEAK_THRESH
MIN_PEAK_THRESH = 35;
## Second, it must be at least SNR_DROP_THRESH lower than the peak SNR in
## PEAK_SNR_THRESH_RADIUS samples
SNR_DROP_THRESH = 20;
PEAK_SNR_THRESH_RADIUS = 2500;
## The threshold is set as follows:
## 1. We look around in a range of PEAK_SNR_THRESH_RADIUS samples around the "signal region"
##    and we find the maximum SNR present in that range.
## 2. We require that the peak SNR of the "high-SNR region" be at most
##   SNR_DROP_THRESH decibels lower than that maximum value.
## 3. We also require that the peak SNR be no lower than MIN_PEAK_THRESH.

In [ ]:
high_snr_locations = findhighsnrregions(snr, SIGNAL_THRESH, MIN_PEAK_THRESH, MAXFILTER_LENGTH,
    snr_drop_thresh=SNR_DROP_THRESH, peak_snr_thresh_radius=PEAK_SNR_THRESH_RADIUS);

Let's see how well the algorithm did! If there is any critical section of the data (e.g., buzz phase vocalizations), you can make sure that the algorithm didn't mess up anywhere.

Note that we will validate these sections later by only keeping chirp sequences where at least 2 microphones agree on the vocalization time (computed using the location data), so it's ok if there are some erroneous chirp segments, or the algorithm thinks some echos are chirp segments.

In [ ]:
audio_idxs = INTERESTING_IDXS;

In [ ]:
plots = Array{Plots.Plot}(undef, 4);
for mic=1:4
    plots[mic] = plotmicdata(audio_idxs, y[:, mic], label="Mic Data", title=(@sprintf "Audio Data for Mic %d" mic));
    plot!(audioindextoms.(audio_idxs), high_snr_locations[audio_idxs, mic]*maximum(y[audio_idxs,mic]), label="High-SNR Regions", color=:black, linewidth=1.2, legend=:bottomright);
end
plot(plots..., layout=grid(2, 2), size=(1500, 600), legend=:bottomright)

In [ ]:
# Find the start and end indices of every high-SNR region
rough_high_snr_idxs_per_mic = Array{Matrix{Int64}}(undef, 4, 1);
for mic=1:4
    rough_high_snr_idxs_per_mic[mic] = findhighsnrregionidxs(snr, mic, SIGNAL_THRESH, MIN_PEAK_THRESH, MAXFILTER_LENGTH,
        snr_drop_thresh=SNR_DROP_THRESH, peak_snr_thresh_radius=PEAK_SNR_THRESH_RADIUS);
end

### Refine the ends of the high-SNR regions

This method more or less gets the starts of the vocalizations, but it might cut off the end too early.

So, we adjust the end of the chirp sequence using the `adjusthighsnridxs` function, which uses a similar algorithm to `findhighsnrregions`, except with a lower SNR threshold and longer max filter size. It also automatically prevents end of the chirp sequence from going past the beginning of the next chirp sequence. 

In [ ]:
### CONFIGURABLE PARAMETERS ###
TAIL_SNR_THRESH=20;
TAIL_MAXFILTER_LENGTH=50; # in samples

In [ ]:
high_snr_idxs_per_mic = Array{Matrix{Int64}}(undef, 4, 1);
for mic=1:4
    high_snr_idxs_per_mic[mic] = copy(rough_high_snr_idxs_per_mic[mic]);

    N_seqs = size(high_snr_idxs_per_mic[mic], 1);
    for row=1:N_seqs
        max_end_idx = (row == N_seqs) ? size(snr, 1) : high_snr_idxs_per_mic[mic][row+1, 1];
        high_snr_idxs_per_mic[mic][row, :] = 
                adjusthighsnridxs(snr, mic, rough_high_snr_idxs_per_mic[mic][row, :],
                    max_end_idx, TAIL_SNR_THRESH, maxfilter_length=TAIL_MAXFILTER_LENGTH);
    end
end

Let's verify that the high-SNR regions produced are reasonable.

The algorithm might pick up a few echos and think that they are chirp sequences. This is unavoidable, and some of these echos should be filtered out in the next section.
It also might cut off the beginning of some chirp sequences if the beginning has very low SNR; we will compensate for this later.

In [ ]:
mic = randint(4);
while size(high_snr_idxs_per_mic[mic], 1) == 0
    mic = randint(4);
end
seq_num = randint(size(high_snr_idxs_per_mic[mic], 1));
(@printf "Mic: %d, high-SNR region number: %d\n" mic seq_num);

bounds = high_snr_idxs_per_mic[mic][seq_num, :];
plotSTFTtime(y[bounds[1]:bounds[2], mic], noverlap=255, zero_pad=true)

## Step 3: Determine chirp sequences that arose from the same initial bat vocalization

### Terminology: Chirp Sequences

A chirp sequence is defined as all microphone outputs that result from a single bat vocalization.

This is a somewhat overloaded term, which can mean one of two things:
1. For a single microphone, a chirp and subsequent echos. We can call this a "single-mic chirp sequence".
2. The chirp and subsequent echos, but for all microphones that picked up the chirp. We can call this a "multi-mic chirp sequence".

### Grouping Single-Mic Chirp Sequences to Make Multi-Mic Chirp Sequences

Now that we know (approximately) where the chirp sequences are, for each microphone, we can group them by which ones came from the same initial chirp. Also, for robustness, we only keep chirp sequences where two or more microphones "agree" on the initial vocalization time, or where the SNR is very high.

The vocalization time is calculated by interpolating the centroid data and solving for `t` in `distance_from_mic(t) = speed_of_sound * (time_chirp_reached_mic - t)`.

### Read in centroid data and microphone locations

We need the centroid data for this section, so let's read it in.

In [ ]:
centroids = matread(CENTROID_FILENAME)[CENTROID_VARIABLE_NAME];
if size(centroids, 1) == 3
    centroids = Matrix(transpose(centroids));
end
centroids[end-9:end, :]

In [ ]:
first_non_nan_idx = findfirst(.~isnan.(centroids[:, 1]));
(@printf "First time where the centroid data is not NaN: %d milliseconds\n" 1000*videoindextosec(first_non_nan_idx, size(centroids, 1)))

In [ ]:
mic_positions = matread(MIC_POSITION_FILENAME)[MIC_POSITION_VARIABLE_NAME];
if size(mic_positions, 1) == 3
    mic_positions = Matrix(transpose(mic_positions));
end
mic_positions

### Run `groupchirpsequencesbystarttime` to determine which chirp sequences came from the same vocalization

In [ ]:
### CONFIGURABLE PARAMETERS ###
TEMPORAL_TOLERANCE_MS = 2; #  if the estimated vocalization times for two
                             # chirp sequences (for different microphones) are
                             # within this number of milliseconds apart, they
                             # are considered to be from the same vocalization.

SINGLE_MIC_SNR_THRESH = 85;  # if a chirp sequence only has data from only one
                             # microphone, still store the chirp sequence if it
                             # has an SNR over this value.

ANY_MIC_SNR_THRESH = 45; # One of the microphones needs to have an SNR over this
                         # threshold, or else it's not a valid chirp sequence
                         # (probably an echo)

In [ ]:
chirp_sequences, vocalization_times = 
        groupchirpsequencesbystarttime(high_snr_idxs_per_mic, snr, y, centroids,
            mic_positions, single_mic_snr_thresh=SINGLE_MIC_SNR_THRESH,
            any_mic_snr_thresh=ANY_MIC_SNR_THRESH,
            vocalization_start_tolerance_ms=TEMPORAL_TOLERANCE_MS);

### Breaking down the return values of `groupchirpsequencesbystarttime`
#### First Output: `chirp_sequences`

We store the multi-mic chirp sequences as an `Array{Dict{Int, ChirpSequence}}`. This seems complicated, so let's break it down:
- Each element of `chirp_sequences` corresponds to a different bat vocalization, i.e., each element of `chirp_sequences` is a multi-mic chirp sequence.
  - Let's zoom in, for instance, on `chirp_sequences[1]`, aka the multi-mic chirp sequence corresponding to the first bat vocalization.
  
    An example of what a multi-mic chirp sequence looks like is `{1 => ChirpSequence(...), 3 => ChirpSequence(...), 4 => ChirpSequence(...)}`
     
     This is a mapping from a microphone number (in this case, the microphones are 1, 3, and 4) to a single-mic chirp sequence.

    - Let's zoom in further to `chirp_sequences[1][4]`. This is the single-mic chirp sequence for what microphone 4 heard from the first bat vocalization.
   
      We represent this by a data struct: `ChirpSequence`. This contains the following fields:
      1. `start_idx`: when the chirp reached the microphone, in number of samples since the beginning of the microphone data _originally_ read in.
      2. `length`: number of audio samples in the single-mic chirp sequence.
      3. `vocalization_time_ms`: approximate time of the bat vocalization, estimated using the onset of the chirp sequence in the microphone data and the distance of the bat to the respective microphone.
      4. `snr_data`: SNR of the microphone data over the duration of the single-mic chirp sequence.
      5. `mic_data`: oscilloscope data for this single-mic chirp sequence.
      6. `mic_num`: microphone that the above data is from.
     
#### Second Output: `vocalization_times`

This is the time, in milliseconds, of each bat vocalization. There is a one-to-one correspondence between indices of `vocalization_times` and indices of `chirp_sequences`.
Each element of `vocalization_times` is the average of the vocalization times estimated for each microphone that heard the vocalization.

### Visualizing `vocalization times`

In [ ]:
println("There were ", length(vocalization_times), " vocalizations!")

In [ ]:
myplot(vocalization_times, ones(length(vocalization_times)), line=:stem, marker=:circle, color=:1, markersize=5,
    title="Vocalizations ", xlabel="Milliseconds", ylabel="", size=(1200, 200), yrange=(0, 1.2), legend=false)

In [ ]:
myplot(vocalization_times[2:end]-vocalization_times[1:end-1], marker=:circle, ylabel="Milliseconds", xlabel="Vocalization Number",
    title="Separation between subsequent vocalizations, in milliseconds (y-axis has a log scale)", yaxis=:log, legend=false)

### Visualizing Chirp Sequences

The following cell plots boxes around chirp sequences, labeled with their estimated vocalization time. Single-mic chirp sequences with the same colored box and same vocalization time belong to the same multi-mic chirp sequence.

In [ ]:
plotchirpsequenceboxes(audioindextoms(INTERESTING_IDXS[1]), audioindextoms(INTERESTING_IDXS[end]), vocalization_times, chirp_sequences, y)

**Note**: If this doesn't look right and you suspect that there is an issue with the centroid data, take a look at the notebook `ValidateCentroidData.ipynb`.

#### Let's plot some random chirp sequences!

_Note_: zeros are added to the end of each single-mic chirp sequence so that they are all the same length. This is just for plotting purposes.

In [ ]:
seq_idx = randint(length(chirp_sequences));
(@printf "Sequence number: %d\n" seq_idx);
plotchirpsequence(chirp_sequences[seq_idx], plot_spectrogram=true)

In [ ]:
# you can plot in the time domain too
plotchirpsequence(chirp_sequences[seq_idx], plot_separate=true)

## Step 4: Estimate the "melody" of the vocalization

Now that we have sufficiently pre-processed the data, we can work towards estimating the original vocalization made by the bat.
We define the "melody" of the vocalization as the _fundmental harmonic_.

The first step is "tracing" the melody:
1. Find the time where the SNR is highest `FIND_HIGHEST_SNR_IN_FIRST_MS` milliseconds since the start of the chirp sequence, and find the melody of the chirp at that point (here, the SNR is hopefully high enough to accurately estimate the melody).
2. Work backwards until the beginning of the chirp, at each index looking for the strongest frequency within some small range of the last frequency found.
3. Repeat, but this time work towards the end of the chirp. To avoid picking up echos, enforce that, once the slope of the melody (with respect to time) becomes negative, it can never become positive.
4. We have reached the end of the chirp when the tone drops off far enough from its maximum value.

After tracing the melody, we need to see if we found the fundamental harmonic
or some higher harmonic. This is done by taking the loudest part of the melody
and dividing the frequency by 2, 3, etc. until we go below 20 kHz. The fundamental harmonic is the lowest such frequency with power at most
`MELODY_DROP_THRESH_DB` below the loudest harmonic.

Also, the function `smoothmelody` uses a moving average filter to produce a smoother version of the melody.

**Note**: `findmelody` returns the fundamental harmonic in indices of the Fourier transform of length-`256` windows of the signal.

`findmelodyhertz` will return the fundamental harmonic in hertz.

In [ ]:
### CONFIGURABLE PARAMETERS ###
MAXIMUM_MELODY_SLOPE = 5; # this is the maximum amount, in Fourier transform
                          # indices, that the melody is allowed to change from
                          # one index to the next.
MELODY_DROP_THRESH_DB = 20; # described above
FIND_HIGHEST_SNR_IN_FIRST_MS = 1.5; # described above

The next cell estimates the melody of a random chirp and plots it on top of the spectrogram in blue. It also plots the estimated end index of the chirp in cyan.

In [ ]:
melody_kwargs = Dict{Symbol, Any}(
    :maximum_melody_slope => MAXIMUM_MELODY_SLOPE,
    :melody_drop_thresh_db => MELODY_DROP_THRESH_DB,
    :find_highest_snr_in_first_ms => FIND_HIGHEST_SNR_IN_FIRST_MS,
    :bandpass_filter => (20_000, 100_000)
);

In [ ]:
findmelodyhertz(seq[mic_idx], MIN_PEAK_THRESH; melody_kwargs...);

In [ ]:
seq_idx = randint(length(chirp_sequences));
seq = chirp_sequences[seq_idx];
mic_idx = collect(keys(seq))[randint(length(seq))];

(@printf "Sequence number: %d, mic: %d\n" seq_idx mic_idx);
melody = findmelody(seq[mic_idx], MIN_PEAK_THRESH; melody_kwargs...);
melody = smoothmelody(melody; filter_size=100);
chirp_bounds = estimatechirpbounds(seq[mic_idx], melody, MIN_PEAK_THRESH);
plotmelody(seq[mic_idx], melody, chirp_bounds)

We find harmonics by searching for the strongest frequency in a range around a multiple of the melody. This is more accurate than just saying that the harmonic is exactly some multiple of the fundamental frequency, especially since the melody might get cut off at lower frequencies.

In [ ]:
harmonic = getharmonic(seq[mic_idx], melody, 2);
harmonic = smoothmelody(harmonic);
plotmelody(seq[mic_idx], harmonic, chirp_bounds)

### Use the melody to approximately separate the chirp from the echos
We can use the strength of the melody to approximately determine when the vocalization ends and the echos begin.

The process is as follows:
1. Find the point where the melody is the strongest.
2. Find the first index, after this point, where the melody strength drops over
    `MELODY_DROP_THRESH_DB` decibels from its peak value (if this cutoff value
    is below `MELODY_THRESH_DB_LOW`, we instead find where the melody strength
    goes below `MELODY_THRESH_DB_LOW`).
    - If the melody strength never drops below this threshold, then just return
        the last index of the chirp sequence.
3. Apply a moving average filter to the melody strength.
4. Apply the following heuristic:
    - The end of the chirp is the first local minimum of the melody strength
    after the index from step 2, or the first time the melody strength dips
    below `MELODY_THRESH_DB_LOW`, whichever comes first.
    - If neither event happens, return the last index of the chirp sequence.

In [ ]:
### CONFIGURABLE PARAMETERS ###
# (parameters are described in the text above)
MELODY_THRESH_DB_LOW = -20;
MOVING_AVG_SIZE = 10;
MELODY_DROP_THRESH_DB_START = 35; # the start index of the chirp is
                                   # computed as the first index where
                                   # the melody strength is greater than the
                                   # maximum melody strength, minus this number.

In [ ]:
chirp_bound_kwargs = Dict{Symbol, Any}(
    :melody_drop_thresh_db => MELODY_DROP_THRESH_DB,
    :melody_thresh_db_low => MELODY_THRESH_DB_LOW,
    :melody_drop_thresh_db_start => MELODY_DROP_THRESH_DB_START,
    :moving_avg_size => MOVING_AVG_SIZE
);
chirp_kwargs = merge(melody_kwargs, chirp_bound_kwargs);

The next cell plots the estimated chirps (for a random chirp sequence) on top of the full chirp sequences. 

In [ ]:
seq_idx = randint(length(chirp_sequences));
(@printf "Sequence number: %d\n" seq_idx);
seq = chirp_sequences[seq_idx];
plotestimatedchirps(seq, MIN_PEAK_THRESH; chirp_kwargs...)

In [ ]:
plotchirpsequence(chirp_sequences[seq_idx], plot_spectrogram=true)

### Aligning Chirps

You may notice that for Pu166/168 data, the beginnings of the chirps are often cut off due to low SNR. The parts that are cut off have such low SNR that we can't do anything with them, but it will matter in Step 5 that chirps from all microphones can be temporally aligned with each other.

So, we zero-pad the beginnings of those sequences such that that they are aligned chirp sequences from the other mics.

You can run the next few cells to see this in action, and look at the documentation for `computemelodyoffsets` for more information. The output of `computemelodyoffsets` is the number of samples cut off from the beginning of each microphone's chirp sequence.

In [ ]:
seq_idx = randint(length(chirp_sequences));
(@printf "Sequence number: %d\n" seq_idx);
offsets = computemelodyoffsets(chirp_sequences[seq_idx], MIN_PEAK_THRESH; chirp_kwargs...)

The following cell plots the chirp sequence for each microphone, zero-padded at the beginning so that the chirps are aligned.

In [ ]:
plotoffsetchirps(chirp_sequences[seq_idx], offsets, getchirpstartandendindices(chirp_sequences[seq_idx], MIN_PEAK_THRESH; chirp_kwargs...)[1])

## Step 5: Use optimization to combine information from all microphones and estimate the actual bat vocalization

We can combine the data from different microphones through **blind deconvolution**, which involves estimating:

1. The actual bat vocalization, $\mathbf{x}$, that produced a given chirp sequence
2. A set of impulse responses $\mathbf{h}_k$ such that, if $\mathbf{y}_k$ is the data from microphone $k$ for the chirp sequence, the convolution relation $\mathbf{x} * \mathbf{h}_k \approx \mathbf{y}_k$ holds.

We don't have enough information to solve the problem, unless we make some assumptions about the vocalization and the impulse response. We assume the following:

1. The impulse responses should be somewhat sparse (most elements are close to zero). Intuatively, it should be nonzero at the beginning of the chirp sequence and at the locations of echos.
2. The melody estimation from the previous section was relatively accurate. So, we can expect most of the energy of the spectrogram of the chirp to be at the melody and its harmonics.

### Utility of this section

The previous sections do pretty well at estimating vocalizations, and the optimization algorithm doesn't do too much better. It can typically remove echos from estimated chirps and clean up some spectral noise, but it does occasionally introduce spectral noise.

This section can be seen as an interesting theoretical extension: formulating echo denoising as a blind deconvolution problem and jointly computing the initial vocalization and the impulse responmse mapping the vocalization to each microphone.

### Optimization problem formulation

Overall, we want to optimize the following things:

1. **Data fitting**: how close $\mathbf{x} * \mathbf{h}_k$ is to the microphone data, $\mathbf{y}_k$, for the given chirp sequence.
2. **Impulse response sparsity**: how many nonzeros $\mathbf{h}_k$ has, approximated (for the sake of the optimization algorithm) by the absolute sum of its elements.
3. **Proximity to estimated melody**: if we don't try to enforce any constraints on what the chirp, $\mathbf{x}$, looks like, the algorithm provides a pretty noisy chirp. So, we constrain that $\mathbf{x}$ should loosely follow the melody we estimated in step 4.
   - *This appears the least reliable part of the optimization objective, mainly because chirps can either be "narrow" or diffuse in frequency.*

You will be able to choose how much each of these three objectives "matter" before performing optimization.

### Data Collection

Let's grab a chirp sequence to perform this optimization on!
We will construct the matrix $\mathbf{Y} \triangleq \begin{bmatrix} \mathbf{y}_{k_1} & \mathbf{Y}_{k_2} & \cdots\end{bmatrix}$ (where $k_1$, $k_2$, etc. are the microphone indices used for this chirp sequence).

_Note_: f the chirp sequences have wildly different amplitudes, the optimization algorithm tends to ignore those with lower amplitudes. So, we normalize each chirp sequence to have an amplitude of 1 when constructing the matrix $\mathbf{Y}$.

In [ ]:
seq_idx = randint(length(chirp_sequences));
chirp_seq = chirp_sequences[seq_idx];
(@printf "Sequence number: %d\n" seq_idx);

pad_len = 300; 
@assert pad_len > 128
offsets = computemelodyoffsets(chirp_seq, MIN_PEAK_THRESH; chirp_kwargs...)
Y, mics = getchirpsequenceY(chirp_seq, offsets, pad_len);
plotchirpsequence(chirp_seq; plot_spectrogram=true)

In [ ]:
plotestimatedchirps(chirp_seq, MIN_PEAK_THRESH; chirp_kwargs...)

### Initial condition

We can give the optimization algorithm a "warm start" using the estimated chirps from Step 4.
To do so, we estimate the chirp and impulse response for each microphone, and choose the microphone that produces the sparsest impulse response.

There are two different methods for finding the initial condition. The first, and **recommended**, is to use `getinitialconditionsnr`, which chooses the microphone with the highest SNR to provide the initial guess for $\mathbf{x}$. The second, using `getinitialconditionsparsity`, is to choose whichever microphone provides the sparsest estimates of the impulse responses.

There is one parameter: `H_FFT_THRESH`: frequency components of the impulse response initial condition at indices where the FFT of `X_init` is less than this threshold will be set to zero. This helps the optimization algorithm converge to a sparse impulse response.

In [ ]:
### CONFIGURABLE PARAMETER ###
H_FFT_THRESH = 0.1;

In [ ]:
X_init, H_init, longest_chirp = getinitialconditionsnr(Y, chirp_seq, mics, MIN_PEAK_THRESH, h_fft_thresh=H_FFT_THRESH; chirp_kwargs...)
plotSTFTtime(X_init; title="Chirp initial condition", nfft=256, noverlap=255)

In [ ]:
plotmicdata(H_init, title="Impulse response initial condition")

### Optimization parameters
You can choose how much to weight each term in the optimization problem: think of these parameters as percentages of how important each component of the optimization problem is, though they need not sum up to 100. The following cell has some reasonable values.

In [ ]:
### CONFIGURABLE PARAMETERS ###

DATA_FITTING_WEIGHT = 70; ## Making H * X close to Y
H_SPARSITY_WEIGHT = 5; ## Looking for sparse impulse responses. Generally, should be weighted pretty low.
MELODY_WEIGHT = 35; ## Maing melody of the chirp close to the result of findmelody

How spaced out should the spectrogram windows be? A good heuristic is to have 50 windows over the length of the chirp.

In [ ]:
STFT_STRIDE = Int64(ceil(longest_chirp / 50));

How long to run the optimization algorithm for? 10000 iterations seems to work pretty well; running it for longer may cause the algorithm to converge better if it doesn't converge well already (presumably, giving better results), and running for fewer iterations makes it faster.

In [ ]:
### CONFIGURABLE PARAMETER ###

MAX_ITER=10000;

How "diffuse" is the melody in frequency? This is determined by a pair of parameters called `MELODY_RADIUS_START` and `MELODY_RADIUS_END`, which are approximations of how diffuse the melody is (approximately, in kilohertz) at the start and end of the chirp (as chirps tend to become narrower at the end).

In [ ]:
MELODY_RADIUS_START = 10; MELODY_RADIUS_END = 0; # NOT BUZZ
# MELODY_RADIUS_START = 15; MELODY_RADIUS_END = 5; # BUZZ

### Optimization

Run the following cell to see what the optimization algorithm thinks the bat vocalization was (along with the impulse responses of each microphone for the given chirp)!

In [ ]:
X_opt, H_opt, max_chirp_len = optimizePALM(chirp_seq, mics, Y, H_init, X_init, MIN_PEAK_THRESH,
                        DATA_FITTING_WEIGHT, H_SPARSITY_WEIGHT, MELODY_WEIGHT;
                        melody_radius_start=MELODY_RADIUS_START, melody_radius_end=MELODY_RADIUS_END,
                        max_iter=MAX_ITER, nfft=256, stft_stride=STFT_STRIDE, chirp_kwargs...);

_Note_: if it is having a hard time finding a sparse impulse response, try setting `H_SPARSITY_WEIGHT` to `50` and increase the number of iterations.

### Optimization Results: Vocalization

In [ ]:
plotmicdata(X_opt, label="x", title="Optimization Output: Time Domain")

#### Play the estimated chirp, slowed down by a factor of 15

In [ ]:
using WAV;
SLOW_DOWN_FACTOR = 15;
wavplay(X_opt[128:max_chirp_len] ./ maximum(abs.(X_opt)), round(FS / SLOW_DOWN_FACTOR));

#### Frequency-Domain Results: optimization algorithm estimation of the vocalization

In [ ]:
plotSTFTtime(X_opt[128:max_chirp_len+128], title="Optimization Output: Vocalization", noverlap=255)

#### For comparison: vocalization initial condition

In [ ]:
plotSTFTtime(X_init[128:max_chirp_len+128], title="Initial Condition: Vocaliztion", noverlap=255)

### Optimization Results: Impulse Responses

**Note** sometimes there might be significant peaks at the *end* of the impulse responses. This is not unexpected! We are actually using the circular convolution of the vocalization and impulse response to estimate the microphone data, so those peaks at the end can be thought of as occuring before the pictured start of the impulse response.

In [ ]:
K=size(Y, 2);
plots = Matrix(undef, 1, 5);
for i=1:K
    plots[i] = plotmicdata(H_opt[:, i], title=(@sprintf "Impulse response for mic %d" mics[i]), legend=false)
end
plots[K+1] = myplot([0, 0], legend=false, title="Blank Plot", xlabel="", ylabel="");

num_rows = Int64(floor(K + K%2) / 2);
plot(plots[1:num_rows*2]..., layout=(num_rows, 2), size=(1100, num_rows*300))

### Optimization Results: Estimation Error

How far is $\mathbf{x} * \mathbf{h}_k$ from the actual microphone data? For the right choise of optimization parameters, the estimation error (orange curve plotted below) should be insignificant (same order of magnitude as noise).

In [ ]:
K=size(Y, 2);
plots = Matrix(undef, 1, 5);
Y_hat = circconv(X_opt, H_opt);
for i=1:K
    plots[i] = plotmicdata([Y[:, i] Y_hat[:, i]-Y[:, i]], title=(@sprintf "Mic data and estimation error for mic %d" mics[i]), label=["Mic data" "Error"])
end
plots[K+1] = myplot([0, 0], legend=false, title="Blank Plot", xlabel="", ylabel="");

num_rows = Int64(floor(K + K%2) / 2);
plot(plots[1:num_rows*2]..., layout=(num_rows, 2), size=(1100, num_rows*300))

In [ ]:
K=size(Y, 2);
plots = Matrix(undef, 1, 5);
for i=1:K
    plots[i] = plotmicdata(Y_hat[:, i], title="Est. chirp convolved with est. impulse response")
end
plots[K+1] = myplot([0, 0], legend=false, title="Blank Plot", xlabel="", ylabel="");

num_rows = Int64(floor(K + K%2) / 2);
plot(plots[1:num_rows*2]..., layout=(num_rows, 2), size=(1100, num_rows*300))